# Regional European Daily Event Analysis: Marine Heatwave ID & Tracking using `MarEx` Regional Mode

### `MarEx` Processing Pipeline for Regional European Datasets:

This example demonstrates **regional tracking** capabilities for high-resolution (0.05°) European marine extremes using `marEx.regional_tracker()`. Regional mode is specifically designed for focused geographical studies where coordinate system auto-detection may fail.

---

1. **Morphological Pre-Processing**
    - Performs binary morphological closing using `dask_image.ndmorph` to fill small spatial holes up to `R_fill` cells in radius 
    - Executes binary opening to remove isolated small features of order `R_fill`
    - Fills gaps in time to maintain event continuity for interruptions up to `T_fill` time steps
    - Filters out objects smaller than `area_filter_absolute` cells

2. **Blob Identification**
    - Labels spatially connected components using efficient connected-component algorithm in `dask_image.ndmeasure`
    - Computes blob properties (area, centroid, boundaries)

3. **Temporal Tracking**
    - Identifies blob overlaps between consecutive time frames
    - Connects objects across time, applying the following criteria for splitting, merging, & persistence:
        - Connected objects must overlap by at least fraction `overlap_threshold` of the smaller area
        - Merged objects retain their original ID, but partition the child area based on the parent of the _nearest-neighbour_ cell (or centroid distance)

4. **Graph Reduction & Finalisation**
    - Constructs the complete temporal graph of object evolution through time
    - Resolves object connectivity graph using `scipy.sparse.csgraph.connected_components`
    - Creates globally unique IDs for each tracked extreme event
    - Maps objects into efficient ID-time space for convenient analysis
    - Computes comprehensive statistics about the lifecycle of each event

The pipeline leverages **dask** for distributed parallel computation, enabling efficient processing of large datasets. \
A 40-year regional European daily (OSTIA) analysis at full 0.05° resolution on 32 cores takes
- Basic (i.e. Scannell et al., which involves no merge/split criteria or tracking):  ~3 minutes
- Full Split/Merge Thresholding & Merge Tracking:  ~15 minutes

In [1]:
from getpass import getuser
from pathlib import Path

import dask
import xarray as xr

import marEx
import marEx.helper as hpc

In [2]:
# Lustre Scratch Directory
scratch_dir = Path("/scratch") / getuser()[0] / getuser()

In [3]:
# Start Dask Cluster
client = hpc.start_local_cluster(
    n_workers=32, threads_per_worker=1, scratch_dir=scratch_dir / "clients"
)  # Specify temporary scratch directory for dask to use

Hostname: l40193
Forward Port: l40193:8787
Dashboard Link: localhost:8787/status


In [4]:
# Choose optimal chunk size & load data
#   N.B.: This is crucial for dask (not only for performance, but also to make the problem tractable)
#         The operations are eventually global-in-space, and so requires the spatial dimension to be contiguous/unchunked
#         We can adjust the chunk size in time depending on available system memory.

chunk_size = {"time": 25, "lat": -1, "lon": -1}

In [5]:
# Load Pre-processed Data (cf. `01_preprocess_extremes.ipynb`)

file_name = scratch_dir / "mhws" / "extremes_binary_regional_shifting_hobday.zarr"
ds = xr.open_zarr(str(file_name), chunks=chunk_size)

In [6]:
# Run Regional ID, Tracking, & Merging using marEx.regional_tracker()

# Key differences from global tracker:
# - Uses marEx.regional_tracker() instead of marEx.tracker()
# - Requires explicit coordinate_units specification
# - Optimised for regional coordinate systems

regional_tracker = marEx.regional_tracker(
    ds.extreme_events,
    ds.mask,
    coordinate_units="degrees",  # Explicit specification required for regional mode
    grid_resolution=0.05,  # Grid resolution in degrees, used to calculate the object areas on the globe
    area_filter_absolute=600,  # Remove objects smaller than 600 cells
    R_fill=16,  # Fill small holes with radius < 16 _cells_
    T_fill=4,  # Allow gaps of 4 days and still continue the event tracking with the same ID
    allow_merging=True,  # Allow extreme events to split/merge. Keeps track of merge events & unique IDs.
    overlap_threshold=0.25,  # Overlap threshold for merging events. If overlap > threshold, events merge, are partitioned, and are independently tracked
    nn_partitioning=True,  # Use new NN method to partition merged children areas. If False, reverts to old method of Di Sun et al. 2023.
    verbose=True,
)

extreme_events_ds, merges_ds = regional_tracker.run(return_merges=True)
extreme_events_ds

Tracking Statistics:
   Binary Hobday to Processed Area Fraction: 0.7047647269550515
   Total Object Area IDed (cells): 331612330.0
   Number of Initial Pre-Filtered Objects: 40405
   Number of Final Filtered Objects: 34597
   Area Cutoff Threshold (cells): 600
   Accepted Area Fraction: 0.997835095576814
   Total Events Tracked: 4564
   Total Merging Events Recorded: 7360


<xarray.Dataset> Size: 42GB
Dimensions:       (time: 9282, lat: 800, lon: 1300, ID: 4564, component: 2,
                   sibling_ID: 15)
Coordinates:
  * time          (time) datetime64[ns] 74kB 1997-01-01T12:00:00 ... 2022-05-...
  * ID            (ID) int32 18kB 1 2 3 4 5 6 ... 4559 4560 4561 4562 4563 4564
  * component     (component) int64 16B 0 1
  * lat           (lat) float32 3kB 30.02 30.08 30.12 ... 69.88 69.93 69.97
  * lon           (lon) float32 5kB -19.98 -19.92 -19.88 ... 44.88 44.92 44.97
Dimensions without coordinates: sibling_ID
Data variables:
    ID_field      (time, lat, lon) int32 39GB dask.array<chunksize=(1, 800, 1300), meta=np.ndarray>
    global_ID     (time, ID) int32 169MB dask.array<chunksize=(1, 4564), meta=np.ndarray>
    area          (time, ID) float32 169MB dask.array<chunksize=(1, 4564), meta=np.ndarray>
    centroid      (component, time, ID) float32 339MB dask.array<chunksize=(1, 1, 4564), meta=np.ndarray>
    presence      (time, ID) bool 42MB dask.array<chunksize=(1, 4564), meta=np.ndarray>
    time_start    (ID) datetime64[ns] 37kB dask.array<chunksize=(4564,), meta=np.ndarray>
    time_end      (ID) datetime64[ns] 37kB dask.array<chunksize=(4564,), meta=np.ndarray>
    merge_ledger  (time, ID, sibling_ID) int32 3GB dask.array<chunksize=(1, 4564, 15), meta=np.ndarray>
Attributes: (12/14)
    allow_merging:               1
    N_objects_prefiltered:       40405
    N_objects_filtered:          34597
    N_events_final:              4564
    R_fill:                      16
    T_fill:                      4
    ...                          ...
    accepted_area_fraction:      0.997835095576814
    preprocessed_area_fraction:  0.7047647269550515
    overlap_threshold:           0.25
    nn_partitioning:             1
    total_merges:                7360
    multi_parent_merges:         3121

In [7]:
merges_ds

<xarray.Dataset> Size: 1MB
Dimensions:        (merge_ID: 7360, parent_idx: 15, child_idx: 15)
Dimensions without coordinates: merge_ID, parent_idx, child_idx
Data variables:
    parent_IDs     (merge_ID, parent_idx) int32 442kB 16 18 -1 -1 ... -1 -1 -1
    child_IDs      (merge_ID, child_idx) int32 442kB 20 34598 -1 -1 ... -1 -1 -1
    overlap_areas  (merge_ID, parent_idx) int32 442kB 14392 1881 -1 ... -1 -1 -1
    merge_time     (merge_ID) datetime64[ns] 59kB 1997-01-06T12:00:00 ... 202...
    n_parents      (merge_ID) int8 7kB 2 2 2 3 3 3 3 3 3 3 ... 2 2 2 2 2 2 2 2 2
    n_children     (merge_ID) int8 7kB 2 2 2 3 3 3 3 3 3 3 ... 2 2 2 2 2 2 2 2 2
Attributes:
    fill_value:  -1

In [8]:
# Save IDed/Tracked/Merged Events to `zarr` for more efficient parallel I/O

file_name = scratch_dir / "mhws" / "extreme_events_merged_regional_shifting.zarr"
extreme_events_ds.to_zarr(file_name, mode="w")

In [9]:
# Save Merges Dataset to netcdf
file_name = scratch_dir / "mhws" / "extreme_events_merged_regional_shifting_merges.nc"
merges_ds.to_netcdf(file_name, mode="w")

### Run Basic Tracking for Comparison
N.B.: This is the current standard method used in the literature, which involves _No_ temporal gap filling, _No_ merging/splitting and _No_ independent event tracking.

In [10]:
# Run Basic Regional Tracking

basic_regional_tracker = marEx.regional_tracker(
    ds.extreme_events,
    ds.mask,
    coordinate_units="degrees",  # Explicit specification required for regional mode
    area_filter_absolute=600,  # Remove objects smaller than 600 cells
    R_fill=16,  # Fill small holes with radius < 16 _cells_
    T_fill=0,  # No temporal hole filling
    allow_merging=False,  # Do not allow extreme events to split/merge. All touching events adopt the same ID forever (after _and_ before (!)).
)

extreme_events_basic_ds = basic_regional_tracker.run()
extreme_events_basic_ds

Tracking Statistics:
   Binary Hobday to Processed Area Fraction: 0.8164409999213844
   Total Object Area IDed (cells): 286139612.0
   Number of Initial Pre-Filtered Objects: 41337
   Number of Final Filtered Objects: 34157
   Area Cutoff Threshold (cells): 600
   Accepted Area Fraction: 0.9982279559392148
   Total Events Tracked: 5513


<xarray.Dataset> Size: 39GB
Dimensions:   (time: 9282, lat: 800, lon: 1300)
Coordinates:
  * time      (time) datetime64[ns] 74kB 1997-01-01T12:00:00 ... 2022-05-31T1...
  * lat       (lat) float32 3kB 30.02 30.08 30.12 30.17 ... 69.88 69.93 69.97
  * lon       (lon) float32 5kB -19.98 -19.92 -19.88 ... 44.88 44.92 44.97
Data variables:
    ID_field  (time, lat, lon) int32 39GB dask.array<chunksize=(1, 800, 1300), meta=np.ndarray>
Attributes:
    allow_merging:               0
    N_objects_prefiltered:       41337
    N_objects_filtered:          34157
    N_events_final:              5513
    R_fill:                      16
    T_fill:                      0
    area_filter_quartile:        0.0
    area_threshold (cells):      600
    accepted_area_fraction:      0.9982279559392148
    preprocessed_area_fraction:  0.8164409999213844

In [11]:
# Save IDed Events to `zarr` for more efficient parallel I/O

file_name = scratch_dir / "mhws" / "extreme_events_basic_regional_shifting.zarr"
extreme_events_basic_ds.to_zarr(file_name, mode="w")